## Exploring the MPC's Isolated Tracklet File
#### Matthew J. Holman


12 November 2017

I am using this notebook as a starting point for the cs182 final project to link the ITF.

The plan is to:

    * Get the ITF records into a reasonable format, with the observatory locations determined, the time converted to jd_utc, jd_ut1, and jd_tdb, and the RA/Dec converted to unit vectors.
    * We need to keep the original format lines around, for later orbit fitting.
    * Separate the tracklets into months between successive full moons.
    * Determine the healpix region for each tracklet
    * Transform the tracklets in each time/healpix block into a local projection
    * Do the linear fit for each tracklet to get the a, b, g, adot, bdot solutions
    * Look for clusters in those solutions

### The NOVAS package

First, get the USNO's python NOVAS package.  We'll need that.

http://aa.usno.navy.mil/software/novas/novas_py/novaspy_intro.php

Just type 

pip install novas

pip install novas_de405

Here's the reference:

Barron, E. G., Kaplan, G. H., Bangert, J., Bartlett, J. L., Puatua, W., Harris, W., & Barrett, P. (2011) “Naval Observatory Vector Astrometry Software (NOVAS) Version 3.1, Introducing a Python Edition,” Bull. AAS, 43, 2011.

### The kepcart library

You will need to make sure you have a copy of the kepcart library.  There is a copy of it on the MPC bitbucket site, with some instructions.

In [1]:
%matplotlib inline
import numpy as np
import scipy.interpolate
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.notebook_repr_html', True)
import math
import kepcart as kc
import healpy as hp
import collections
import astropy
from collections import defaultdict
import MPC_library

In [2]:
Observatories = MPC_library.Observatories

In [3]:
ObservatoryXYZ = Observatories.ObservatoryXYZ

In [7]:
Observatories.getObservatoryPosition('F51', 2457000.5)

array([ 0.22807555,  0.87911034,  0.38114111])

Removed the 2-line observations, for now.

In [8]:
Observatories.getObservatoryPosition('C51', 2457000.5)

(None, None, None)

### Reading the MPC observation files

Dealing with files line by line in python is not fast.  

The itf.txt, NumObs.txt, and UnnObs.txt files have a mix of 1-line and 2-line formats.  


In [9]:
# This routine checks the 80-character input line to see if it contains a special character (S, R, or V) that indicates a 2-line 
# record.
def is_two_line(line):
    note2 = line[14]
    return note2=='S' or note2=='R' or note2=='V'

In [10]:
# This routine opens and reads filename, separating the records into those in the 1-line and 2-line formats.
# The 2-line format lines are merged into single 160-character records for processing line-by-line.
def split_MPC_file(filename):
    filename_1_line = filename.rstrip('.txt')+"_1_line.txt"
    filename_2_line = filename.rstrip('.txt')+"_2_line.txt"
    with open(filename_1_line, 'w') as f1_out, open(filename_2_line, 'w') as f2_out:
        line1=None
        with open(filename, 'r') as f:
            for line in f:
                if is_two_line(line):
                    line1=line
                    continue
                if line1 != None:
                    merged_lines = line1.rstrip('\n') + line
                    f2_out.write(merged_lines)
                    line1 = None
                else:
                    f1_out.write(line)
                    line1 = None


Now split the files for the three main MPC observation files.  (Need to convert the section below to code from markdown, if you haven't run it already.)

split_MPC_file('itf_new.txt')

Below are routines that read the files after they have been split into their respective formats.  

In [11]:
def readMPC_1_line(filename='NumObs_1_line.txt', nrows=1000000):
    colspecs = [(0, 5), (5, 12), (12, 13), (13, 14), (14, 15), (15, 32), (32, 44), (44, 56), (65, 71), (77, 80)]
    colnames = ['objName', 'provDesig', 'disAst', 'note1', 'note2', 'dateObs', 'RA', 'Dec', 'MagFilter', 'obsCode']
    df = pd.read_fwf(filename, colspecs=colspecs, names=colnames, header=None, nrows=nrows)
    return df

In [12]:
def convertObs80(line):
    objName   = line[0:5]
    provDesig = line[5:12]
    disAst    = line[12:13]
    note1     = line[13:14]
    note2     = line[14:15]
    dateObs   = line[15:32]
    RA        = line[32:44]
    Dec       = line[44:56]
    mag       = line[65:70]
    filt      = line[70:71]
    obsCode   = line[77:80]
    return objName, provDesig, disAst, note1, note2, dateObs, RA, Dec, mag, filt, obsCode


In [13]:
def splitMagFilter(magFilter):
    pieces = magFilter.split()
    if len(pieces)==0:
        return None, None
    elif len(pieces)==1:
         return pieces[0], None
    else:
        return pieces[0], pieces[1]

        

Here I am generating a version of the MPC data that includes just the few string-like fields at the start of each line, followed by jd_tdb, the unit vector to the observed target, and the heliocentric position vector of the observatory at that time.

CS0051 had a typo on the magnitude, a missing '.' in the number.  I corrected that manually.

We need to preserve tracklets across the time and space boundaries.  That means that the division into time slices and spatial regions should be based on the first member of a tracklet only.  That way the full tracklet is accepted or not, rather than pieces occasionally.

In [14]:
def get_sorted_tracklets(itf_filename):
    tracklets = defaultdict(list)
    tracklets_jd_dict = {}
    with open(itf_filename) as infile:
        for line in infile:
            if not line.startswith('#'):
                desig = line[0:12]
                jd_tdb = float(line[43:57])
                if desig not in tracklets_jd_dict:
                    # Got a new tracklet
                    tracklets_jd_dict[desig] = jd_tdb
                tracklets[desig].append(line)
    sortedTrackletKeys = sorted(tracklets.keys(), key=lambda k: tracklets_jd_dict[k]) 
    return tracklets, tracklets_jd_dict, sortedTrackletKeys

In [15]:
tracklets, tracklets_jd_dict, sortedTracklets = get_sorted_tracklets('itf_new_1_line.mpc')

for k in sortedTracklets[:10]:
    print(k, tracklets_jd_dict[k])    

len(sortedTracklets)

    JH012    2433421.986183
    JUAN067  2433868.078846
    ML51000  2433926.060067
    q00O07S  2434036.06343
    GVWFM1   2434271.92449
    APS2     2434413.993494
    APS1     2434413.993494
    q00WI0O  2435134.903228
    q80A00B  2435207.861012
    RH01     2435572.063148


3884178

In [40]:
UnnObs_tracklets, UnnObs_tracklets_jd_dict, UnnObs_sortedTracklets = get_sorted_tracklets('UnnObs_Training_1_line_A.mpc')

for k in UnnObs_sortedTracklets[:10]:
    print(k, UnnObs_tracklets_jd_dict[k])    

len(UnnObs_sortedTracklets)

K14R17Q_001  2434686.899433
PLS6331_001  2437230.019837
K04R80Q_001  2437230.096228
PLS4702_001  2437230.096228
K04R80Q_002  2437230.999708
PLS6331_002  2437231.964299
PLS4702_002  2437232.02403
PLS4702_003  2437233.051132
K04R80Q_003  2437234.016413
PLS6331_003  2437234.016413


243721

In [16]:
# Returns the jd of new moon, to the nearest half day
def lunation_center(n, tref=2457722.0125, p=29.49791666663562):
    t = tref + p*n
    tp = np.floor(t) + 0.5
    return tp

lunation_center(-900)

2431173.5

Sweep through the tracklets once, outputting them into a sequence of overlapping time ranges.

Set n_begin to a small value
Read a tracklet, which are in ascending date order.

If the date for the tracklet is bigger than the upper date value for n_begin, advance n_begin until the date of the tracklet is withi the upper date value. 

for n in range(n_begin, n_end), 
if the date for the tracklet is within the date range corresponding to time center n, then processing the tracklet using time center n.


In [38]:
def separate_time_windows(tracklets, sortedTracklets, tracklets_jd_dict, file_stem='itf_new_1_line.mpc', n_begin=-825, n_end=14, dt=45.):
    t_center = lunation_center(n_begin)
    files = {}

    header='#trackletID yr   mn dy      obsCode mag filter  jd_tdb       x_target     y_target     z_target      x_obs       y_obs        z_obs     '

    for desig in sortedTracklets:
        jd_tdb = tracklets_jd_dict[desig]
        while(jd_tdb>t_center+dt):
            if n_begin in files:
                files[n_begin].close()
            n_begin +=1
            t_center = lunation_center(n_begin)
        for n in range(n_begin, n_end):
            if jd_tdb<lunation_center(n)-dt:
                break
            if n not in files:
                outfile = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+'.mpc'
                files[n] = open(outfile, 'w')
                files[n].write(header+'\n')
            for line in tracklets[desig]:
                files[n].write(line)


In [ ]:
separate_time_windows(tracklets, sortedTracklets, tracklets_jd_dict)

In [ ]:
separate_time_windows(tracklets, sortedTracklets, tracklets_jd_dict, dt=15.)

In [41]:
separate_time_windows(UnnObs_tracklets, UnnObs_sortedTracklets, UnnObs_tracklets_jd_dict, file_stem='UnnObs_Training_1_line_A.mpc')

In [17]:
# This returns the topocentric distances and new heliocentric
# position vectors to the target, given the assumed distance
# r and the position vector of the observatory re.
def adjust_position(r, rho_target, re):
    rho_x, rho_y, rho_z = rho_target
    xe, ye, ze = re
    Robs = np.sqrt(xe * xe + ye * ye + ze * ze)
    cos_phi = -(rho_x * xe + rho_y * ye + rho_z * ze) / Robs
    phi = np.arccos(cos_phi)
    sin_phi = np.sin(phi)

    xx2 = r*r - Robs*sin_phi * Robs*sin_phi
    
    if xx2 < 0:
        None, None

    xx = np.sqrt(xx2)
    yy = Robs * cos_phi
    
    rho_p = yy + xx

    # This could be done with numpy arrays
    x_p = xe + rho_p*rho_x
    y_p = ye + rho_p*rho_y
    z_p = ze + rho_p*rho_z
    
    rho_m = yy - xx
    
    # This could be done with numpy arrays    
    x_m = xe + rho_m*rho_x
    y_m = ye + rho_m*rho_y
    z_m = ze + rho_m*rho_z
        
    return (rho_p, (x_p, y_p, z_p)), (rho_m, (x_m, y_m, z_m))


In [20]:
# Does the transformations on the data using the date of the n-th new
# moon as the reference time.
#
# It is reading and processing the entire *.mpc file.
#
# This does the heliocentric tranformation for the assumed radius function,
# r_func.
# It then does light-time correction.
# direction.  This is ok for the time span we are considering.
# 
# This generates a file called *.trans, and it incorporates
# the distance assumed in the file name.
#
def transform_positions(n, r_func, file_stem='itf_new_1_line.mpc', dt=45.):
    infilename = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+'.mpc'
    try:
      open(infilename, 'r')
    except IOError:
      return 0    
    t_ref = lunation_center(n)
    r_ref = r_func(t_ref)
    r_name = "_r%.1lf" % (r_ref)
    outfilename = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+r_name+'.trans'
    
    nside = 32
    
    #rot_mat = MPC_library.rotate_matrix(-MPC_library.Constants.ecl)
    
    with open(infilename, 'r') as infile, open(outfilename, 'w') as outfile:
        for line in infile:
            if line.startswith('#'): 
                header = line.rstrip()
                outfile.write(header + '          dt         x_cor       y_cor        z_cor       pix \n')
            else:
                lineID = line[:43]

                jd_tdb = float(line[43:57])

                x_target, y_target, z_target = line[57:97].split()
                r_target = np.array([float(x_target), float(y_target), float(z_target)])

                x_obs, y_obs, z_obs = line[97:135].split()            
                r_obs = np.array([float(x_obs), float(y_obs), float(z_obs)])

                # Adjust positions
                dt = 0.0
                r_prev = r_func(jd_tdb-dt)
                rho_r_p, rho_r_m = adjust_position(r_prev, r_target, r_obs)
                dt = rho_r_p[0]/MPC_library.Constants.speed_of_light
                
                # Do light-time iterations.
                # Probably don't need to do this at this point, because it is
                # being re-done in a later step.  Removing this will remove
                # an iterative step.                
                ''''
                i=0
                while(np.abs(r_func(jd_tdb-dt)-r_prev)>1e-8):
                    rho_r_p, rho_r_m = adjust_position(r_prev, r_target, r_obs)
                    dt = rho_r_p[0]/MPC_library.Constants.speed_of_light
                    r_prev = r_func(jd_tdb-dt)
                    i += 1
                '''

                x, y, z = rho_r_p[1]
                
                # At this point, I think I should fit a line 
                # to the x, y, z components of each tracklet,
                # vs jd_tdb-dt-t_ref
                # rather than waiting for a later processing stage.
                # Then I'll have both a position and velocity vector.
            
                # Below is a simple gravitational acceleration correction
                # that I found to be completely unimportant.
                '''
                r2 = x*x + y*y + z*z
                r3 = r2*np.sqrt(r2)
                fac = MPC_library.Constants.GMsun/r3
                acc_x, acc_y, acc_z = -fac*x, -fac*y, -fac*z
                t_elap = jd_tdb-dt - t_ref
                dx = 0.5*acc_x*t_elap*t_elap
                dy = 0.5*acc_y*t_elap*t_elap
                dz = 0.5*acc_z*t_elap*t_elap
            
                #pos = np.array([x-dx, y-dy, z-dz])
                
                # Could rotate to ecliptic coordinates
                
                #ecl_pos = np.dot(rot_mat, pos.T).T
                #xp, yp, zp = ecl_pos
                
                xp, yp, zp = x-dx, y-dy, z-dz
                '''
                
                xp, yp, zp = x, y, z
                
                pix = hp.vec2pix(nside, xp, yp, zp, nest=True)
            
                outstring = line.rstrip() + " %13.6lf %12.7lf %12.7lf %12.7lf %5d\n"% \
                      (dt, xp, yp, zp, pix)

                outfile.write(outstring)

        

In [21]:
## The time required is mostly file I/O
for n in range(-825,14):
    transform_positions(n, lambda t: 2.5 + 0e-4*(t-lunation_center(10)), dt=45.)

In [31]:
## The time required is mostly file I/O
for n in range(-825,14):
    transform_positions(n, lambda t: 2.5 + 0e-4*(t-lunation_center(10)), dt=15.)

In [42]:
for n in range(-825,14):
    transform_positions(n, lambda t: 2.5 + 0e-4*(t-lunation_center(10)), file_stem='UnnObs_Training_1_line_A.mpc', dt=45.)

In [ ]:
transform_tracklets(tracklets, sortedTracklets, tracklets_jd_dict)

In [ ]:
for n in range(-20,15):
    print(n, lunation_center(n), MPC_library.getEarthPosition(lunation_center(n)))

In [ ]:
transform_tracklets(UnnObs_tracklets, UnnObs_sortedTracklets, UnnObs_tracklets_jd_dict, file_stem='UnnObs_Training_1_line_A.mpc')

In [23]:
def xyz_to_proj_matrix(r_ref):
    x_ref, y_ref, z_ref = r_ref
    r = np.sqrt(x_ref*x_ref + y_ref*y_ref + z_ref*z_ref)
    lon0 = np.arctan2(y_ref, x_ref)
    lat0 = np.arcsin(z_ref/r)
    slon0 = np.sin(lon0)
    clon0 = np.cos(lon0)
    slat0 = np.sin(lat0)
    clat0 = np.cos(lat0)

    mat = np.array([[-slon0, clon0, 0], 
                    [-clon0*slat0, -slon0*slat0, clat0], 
                    [clon0*clat0, slon0*clat0, slat0 ]])
    
    return mat


In [25]:
# This is reading the positions from the *.trans file that are within angle angDeg
# of the vector vec.
# It's saving the transformed lines of each tracklet into a dictionary.
# This function should be doing the fitting too.
#
def select_positions(t_ref, r_func, vec, angDeg, lines, outfilename):
    r_ref = r_func(t_ref)
    r_name = "_r%.1lf" % (r_ref)

    rot_mat = MPC_library.rotate_matrix(-MPC_library.Constants.ecl)

    results_dict = defaultdict(list)
    
    vec = np.array(vec)
    ecl_vec = np.dot(rot_mat, vec.T).T
    vec = ecl_vec    
    vec = vec/np.linalg.norm(vec)
    mat = xyz_to_proj_matrix(vec)
    
    for line in lines:
        if line.startswith('#'): 
            header = line.rstrip()
            #outfile.write(header + '   desig       x_cor       y_cor        z_cor    dt \n')
        else:
            lineID = line[:43]
            trackletID = line[0:12]
                
            jd_tdb = float(line[43:57])
            dtp = float(line[139:150])
            x_cor, y_cor, z_cor = line[150:188].split()
            r_cor = np.array([float(x_cor), float(y_cor), float(z_cor)])
                
            # Rotate to ecliptic coordinates
            ecl_pos = np.dot(rot_mat, r_cor.T).T
            r_cor = ecl_pos

            if np.arccos(np.dot(vec, r_cor)/np.linalg.norm(r_cor))<angDeg*np.pi/180.:
                xt, yt, zt = np.dot(mat, r_cor)/np.linalg.norm(r_cor)
                results_dict[trackletID].append((jd_tdb-dtp, xt, yt, zt))

    mxs, cxs, mys, cys = [], [], [], []
    with open(outfilename, 'w') as outfile:
        outstring = '#  radius = %lf\n' % (r_ref)
        outfile.write(outstring)
        outstring = '#  angDeg = %lf\n' % (angDeg)
        outfile.write(outstring)
        outstring = '#  vec= %lf, %lf, %lf\n' % (vec[0], vec[1], vec[2])
        outfile.write(outstring)
        outfile.write('#  desig              alpha         alpha_dot       beta             beta_dot         dt \n')
        for k, v in results_dict.items():  
            t = [obs[0]-t_ref for obs in v]
    
            x = [obs[1] for obs in v]
            A = np.vstack([t, np.ones(len(t))]).T
            mx, cx = np.linalg.lstsq(A, x)[0]
            y = [obs[2] for obs in v]
            my, cy = np.linalg.lstsq(A, y)[0]
    
            mxs.append(mx)
            cxs.append(cx)
            mys.append(my)
            cys.append(cy)
    
            outstring = "%12s %16.9lf %16.9lf %16.9lf %16.9lf %16.9lf\n" % (k, cx, mx, cy, my, t[0])
            outfile.write(outstring)


In [26]:
hp_dict = defaultdict(list)
infilename = 'itf_new_1_line_2457987.5_pm45.0_r2.5.trans'
with open(infilename) as file:
    for line in file:
        if line.startswith('#'):
            continue
        pix = int(line.split()[-1])
        hp_dict[pix].append(line)
                
nside=8
angDeg=5.0
for i in range(hp.nside2npix(nside)):
    vec = hp.pix2vec(nside, i, nest=True)
    neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
    lines = []
    for pix in neighbors:
        for line in hp_dict[pix]:
            lines.append(line)
    outfilename = infilename.rstrip('.trans') + '_hp_' + str(i)
    select_positions(lunation_center(9), lambda x: 2.5, vec, angDeg, lines, outfilename)
    #make_figure(outfilename)
    
        


In [33]:
# Trying a slightly different approach.
# The set of lines that are being passed in have
# been selected to be in the same region of sky
# for an assumed distance.  We are going to re-transform
# those assuming a fixed z (or gamma) value with respect
# to the sun and the reference direction, rather than a 
# fixed r, at the reference time
# 
# Rotate observatory positions to projection coordinates, 
# and recalculate simple z-based light-time correction.
#
# Rotate the observations to projection coordinates,
# but they will be theta_x, theta_y only
#
# Fit the simple abg model, for fixed gamma and
# possibly gamma_dot.
#
def select_positions_z(t_ref, z_func, vec, angDeg, lines, outfilename):
    z_ref = z_func(t_ref)
    z_name = "_z%.1lf" % (z_ref)

    rot_mat = MPC_library.rotate_matrix(-MPC_library.Constants.ecl)

    results_dict = defaultdict(list)
    
    vec = np.array(vec)
    ecl_vec = np.dot(rot_mat, vec.T).T
    vec = ecl_vec    
    vec = vec/np.linalg.norm(vec)
    mat = xyz_to_proj_matrix(vec)
    
    sep2 = angDeg*np.pi/180. * angDeg*np.pi/180.
    
    for line in lines:
        if line.startswith('#'): 
            header = line.rstrip()
        else:
            lineID = line[:43]
            trackletID = line[0:12]
                
            jd_tdb = float(line[43:57])
            dtp = float(line[139:150])
            
            # Get unit vector to target
            x_target, y_target, z_target = line[58:98].split()
            r_target = np.array([float(x_target), float(y_target), float(z_target)])
            
            # Rotate to ecliptic coordinates
            r_target_ec = np.dot(rot_mat, r_target.T).T
            
            # Rotate to projection coordinates
            theta_x, theta_y, theta_z = np.dot(mat, r_target_ec)
            # Ignore theta_z after this; it should be very nearly 1.
            
            if (theta_x*theta_x + theta_y*theta_y) < sep2:

                # Get observatory position
                x_obs, y_obs, z_obs = line[98:138].split()
                r_obs = np.array([float(x_obs), float(y_obs), float(z_obs)])
            
                # Rotate to ecliptic coordinates
                r_obs_ec = np.dot(rot_mat, r_obs.T).T
            
                # Rotate to projection coordinates            
                xe, ye, ze = np.dot(mat, r_obs_ec)
            
                dlt = ze/MPC_library.Constants.speed_of_light
                
                results_dict[trackletID].append((jd_tdb, dlt, theta_x, theta_y, theta_z, xe, ye, ze))

    mxs, cxs, mys, cys = [], [], [], []
    with open(outfilename, 'w') as outfile:
        outstring = '#  z_ref = %lf\n' % (z_ref)
        outfile.write(outstring)
        outstring = '#  angDeg = %lf\n' % (angDeg)
        outfile.write(outstring)
        outstring = '#  vec= %lf, %lf, %lf\n' % (vec[0], vec[1], vec[2])
        outfile.write(outstring)
        outfile.write('#  desig              alpha         alpha_dot       beta             beta_dot         dt \n')
        for k, v in results_dict.items():  

            t_emit = [obs[0]-obs[1]-t_ref for obs in v]
            A = np.vstack([t_emit, np.ones(len(t_emit))]).T
            
            x = [obs[2] - obs[5]/z_func(obs[0]-t_ref) for obs in v]
            mx, cx = np.linalg.lstsq(A, x)[0]
            
            y = [obs[3] - obs[6]/z_func(obs[0]-t_ref) for obs in v]
            my, cy = np.linalg.lstsq(A, y)[0]
    
            mxs.append(mx)
            cxs.append(cx)
            mys.append(my)
            cys.append(cy)
    
            outstring = "%12s %16.9lf %16.9lf %16.9lf %16.9lf %16.9lf\n" % (k, cx, mx, cy, my, t_emit[0])
            outfile.write(outstring)



In [28]:
hp_dict = defaultdict(list)
infilename = 'itf_new_1_line_2457987.5_pm45.0_r2.5.trans'
with open(infilename) as file:
    for line in file:
        if line.startswith('#'):
            continue
        pix = int(line.split()[-1])
        hp_dict[pix].append(line)
                
nside=8
angDeg=5.5
for i in range(hp.nside2npix(nside)):
    vec = hp.pix2vec(nside, i, nest=True)
    neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
    lines = []
    for pix in neighbors:
        for line in hp_dict[pix]:
            lines.append(line)
    outfilename = infilename.rstrip('.trans') + '_hp_' + str(i)
    select_positions_z(lunation_center(9), lambda x: 2.5, vec, angDeg, lines, outfilename)
    #make_figure(outfilename)
    
        



Now that I have a set of tracklets that are divided into healpix regions based on a heliocentric transformation,
I can still scan through a set of z values for that selection.

In [55]:
hp_dict = defaultdict(list)
infilename = 'itf_new_1_line_2457987.5_pm45.0_r2.5.trans'
with open(infilename) as file:
    for line in file:
        if line.startswith('#'):
            continue
        pix = int(line.split()[-1])
        hp_dict[pix].append(line)

n=9
nside=8
angDeg=5.5
z=2.4
for i in range(hp.nside2npix(nside)):
    vec = hp.pix2vec(nside, i, nest=True)
    neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
    lines = []
    for pix in neighbors:
        for line in hp_dict[pix]:
            lines.append(line)
    outfilename = infilename.rstrip('.trans') + '_z'+str(z)+'_hp_' + str(i)
    select_positions_z(lunation_center(n), lambda x: z, vec, angDeg, lines, outfilename)
    #make_figure(outfilename)
    
        




In [ ]:
hp.nside2npix(nside)
nside, 1.83*32/8*np.sqrt(2)/2


In [29]:
def make_figure(filename):
    plt.ioff()
    mxs, cxs, mys, cys =[], [], [], []
    for line in open(filename):
        if line.startswith('#'):
            continue
        desig, cx, mx, cy, my, dt = line.split()
        mxs.append(float(mx))
        cxs.append(float(cx))
        mys.append(float(my))
        cys.append(float(cy))
    
    fig=plt.figure(figsize=(18, 16))
    plt.quiver(cxs, cys, mxs, mys, scale=1.0, width=0.0003)
    plt.xlim(-0.2, 0.2)
    plt.ylim(-0.2, 0.2)
    plt.xlabel('alpha')
    plt.ylabel('beta')
    outfile = filename+'.pdf'
    plt.savefig(outfile)
    plt.close()

In [30]:
make_figure('itf_new_1_line_2457987.5_pm45.0_r2.5_hp_764')

In [34]:
make_figure('itf_new_1_line_2457987.5_pm45.0_r2.5_hp_759')

In [35]:
make_figure('itf_new_1_line_2457987.5_pm45.0_r2.5_hp_765')

In [36]:
make_figure('itf_new_1_line_2457987.5_pm45.0_r2.5_hp_766')

In [51]:
n=-12
hp_dict = defaultdict(list)
infilename = 'UnnObs_Training_1_line_A_2457368.5_pm45.0_r2.5.trans'
with open(infilename) as file:
    for line in file:
        if line.startswith('#'):
            continue
        pix = int(line.split()[-1])
        hp_dict[pix].append(line)
                
nside=8
angDeg=5.5
for i in range(hp.nside2npix(nside)):
    vec = hp.pix2vec(nside, i, nest=True)
    neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
    lines = []
    for pix in neighbors:
        for line in hp_dict[pix]:
            lines.append(line)
    outfilename = infilename.rstrip('.trans') + '_hp_' + str(i)
    select_positions_z(lunation_center(n), lambda x: 2.5, vec, angDeg, lines, outfilename)
    #make_figure(outfilename)
    
        

In [52]:
make_figure('UnnObs_Training_1_line_A_2457368.5_pm45.0_r2.5_hp_367')

In [49]:
lunation_center(-12)

2457368.5